In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/BOAZ/분석 adv

/content/drive/MyDrive/BOAZ/분석 adv


In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 데이터 조작 관련
import pandas as pd
import numpy as np
import seaborn as sns
import re
import io

# 한국어 nlp
from konlpy.tag import *   # 모든 형태소분석기 import 하기
#1. hannanum = Hannanum()
#2. kkma = Kkma()
#3. komoran = Komoran()
#4. okt = Okt()

okt = Okt()

### 위 4개중 원하는 형태소분석기를 사용하면 됨 --> okt추천

# 영어 nlp 관련
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from collections import Counter
from PIL import Image

# 기타
from tqdm import tqdm_notebook, tqdm   # for문 진행상황 눈으로 확인 (loading bar)
import datetime

참고사이트: https://wikidocs.net/30707

# topic modeling

# wordcloud

In [60]:
df1= pd.read_excel('2022-08-16_1위 musinsa crawling.xlsx')#,sheet_name = None)
#top1.to_csv("2022-08-16_1위 musinsa crawling.csv")
df1.head()

,profile,item,content,evaluation
0,"\n남성, 175cm, 68kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티",\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
1,"\n남성, 174cm, 68kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당,\n\n사이즈 커요\n밝기 보통이에요\n색감 보통이에요\n두께감 얇아요\n\n
2,"\n남성, 180cm, 76kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
3,"\n남성, 168cm, 66kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
4,"\n여성, 178cm, 58kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 보통이에요\n\n


In [61]:
df2 = pd.DataFrame(df1['profile'].str.split(',').tolist(),columns=['gender','height','weight'])
df2.head()


,gender,height,weight
0,\n남성,175cm,68kg\n신고\n
1,\n남성,174cm,68kg\n신고\n
2,\n남성,180cm,76kg\n신고\n
3,\n남성,168cm,66kg\n신고\n
4,\n여성,178cm,58kg\n신고\n


In [62]:
#df2['gender'] = df2['gender'].str.replace('￦n', '') #regex=True)
df2["gender"] = df2["gender"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df2["weight"] = df2["weight"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
df2["weight"] = df2["weight"].str.replace(pat=r'신고', repl=r'', regex=True)
df2

,gender,height,weight
0,남성,175cm,68kg
1,남성,174cm,68kg
2,남성,180cm,76kg
3,남성,168cm,66kg
4,여성,178cm,58kg
...,...,...,...
595,남성,171cm,69kg
596,남성,173cm,69kg
597,남성,188cm,92kg
598,신고,None,None


In [63]:
df = pd.concat([df2, df1], axis = 1)
df.head()

,gender,height,weight,profile,item,content,evaluation
0,남성,175cm,68kg,"\n남성, 175cm, 68kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티",\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
1,남성,174cm,68kg,"\n남성, 174cm, 68kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당,\n\n사이즈 커요\n밝기 보통이에요\n색감 보통이에요\n두께감 얇아요\n\n
2,남성,180cm,76kg,"\n남성, 180cm, 76kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
3,남성,168cm,66kg,"\n남성, 168cm, 66kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
4,여성,178cm,58kg,"\n여성, 178cm, 58kg\n신고\n",에센셜 쿨 코튼 2-PACK 티셔츠,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 보통이에요\n\n


In [64]:
df.drop(columns=["profile"],inplace=True)
df

,gender,height,weight,item,content,evaluation
0,남성,175cm,68kg,에센셜 쿨 코튼 2-PACK 티셔츠,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티",\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
1,남성,174cm,68kg,에센셜 쿨 코튼 2-PACK 티셔츠,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당,\n\n사이즈 커요\n밝기 보통이에요\n색감 보통이에요\n두께감 얇아요\n\n
2,남성,180cm,76kg,에센셜 쿨 코튼 2-PACK 티셔츠,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 얇아요\n\n
3,남성,168cm,66kg,에센셜 쿨 코튼 2-PACK 티셔츠,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
4,여성,178cm,58kg,에센셜 쿨 코튼 2-PACK 티셔츠,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~,\n\n사이즈 보통이에요\n밝기 밝아요\n색감 선명해요\n두께감 보통이에요\n\n
...,...,...,...,...,...,...
595,남성,171cm,69kg,에센셜 쿨 코튼 2-PACK 티셔츠,일단 핏이 맘에 듭니다 딱 적당한 세미오버핏으로좋고 반바지에 꺼내입거나 청바지 이쁜...,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
596,남성,173cm,69kg,에센셜 쿨 코튼 2-PACK 티셔츠,1+1 인데 옷도 좋네 머하는짓이죠 진짜 완전 감사드립니다 난 너무 만족합니다,\n\n사이즈 보통이에요\n밝기 보통이에요\n색감 보통이에요\n두께감 얇아요\n\n
597,남성,188cm,92kg,쿨 코튼 2-PACK 티셔츠 화이트,"기본템으로 딱입니다. 목이 너무 좁은 티를 싫어하는데 요건 목이 조이는느낌도 없고,...",\n\n사이즈 커요\n밝기 보통이에요\n색감 보통이에요\n두께감 보통이에요\n\n
598,신고,None,None,에센셜 쿨 코튼 2-PACK 티셔츠,흰티에 까만 트레이닝 바지 최고…편의점이나 가까운곳 갈때 막입기 좋아옹,\n\n사이즈 커요\n밝기 보통이에요\n색감 보통이에요\n두께감 얇아요\n\n


In [65]:
# 쓸모있는 컬럼만 뽑아내기
df = df.loc[ : ,['content']]
df.head()

,content
0,"데일리티, 마실, 잠옷 언제든 부담없이 편하게 입을 수 있는 기본 티"
1,오버핏으로 나와서 속에 입기도 좋고 시원하게 입기도 좋습니당
2,하얀색은 조금 비치긴해도 검정색은 안비치고 얇고 시원해서 여름에 입기 좋아요
3,티 두쌍 괜춘해요 시원하고 갠덕으론 검정이 더 이쁘네요
4,살짝오버핏 스타일로 입기 좋구 편하고 예쁘네요~~


In [67]:
num = df['content'].unique()
print(len(num))

599


In [69]:
a=0

for i in range(len(num)):

    for k in df['content']:
        try:            
            nouns[a].append(okt.nouns(k))  #형태소분석기로 명사 추출!!
        except:
            continue
        
    a=a+1

In [72]:
print(a)

599
